In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [18]:
file_path = './car_evaluation.csv'
df = pd.read_csv(file_path)

print("=== 결측치 현황 ===")
print(df.isnull().sum())

print("\n=== 레이블 분포 ===")
print(df['unacc'].value_counts())

=== 결측치 현황 ===
vhigh      0
vhigh.1    0
2          0
2.1        0
small      0
low        0
unacc      0
dtype: int64

=== 레이블 분포 ===
unacc
unacc    1209
acc       384
good       69
vgood      65
Name: count, dtype: int64


In [20]:
# 레이블 인코딩
label_encoder = LabelEncoder()
encoded_df = df.copy()
for column in df.columns:
    encoded_df[column] = label_encoder.fit_transform(df[column])

# 특성과 레이블 분리
X = encoded_df.drop('unacc', axis=1)
y = encoded_df['unacc']
# 훈련 및 테스트 세트 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
# X_train, X_test, y_train, y_test의 shape 확인
print("\n=== 데이터셋 Shape ===")
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


=== 데이터셋 Shape ===
X_train shape: (1381, 6)
X_test shape: (346, 6)
y_train shape: (1381,)
y_test shape: (346,)


In [24]:
#dt
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)
dt_accuracy = accuracy_score(y_test, y_pred_dt)
print("Decision Tree Accuracy:", dt_accuracy)

#rf
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", rf_accuracy)

#svm
svm_model = SVC(random_state=42)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, y_pred_svm)
print("SVM Accuracy:", svm_accuracy)
#lr
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
lr_accuracy = accuracy_score(y_test, y_pred_lr)
print("Logistic Regression Accuracy:", lr_accuracy)

# KNN
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)
knn_accuracy = accuracy_score(y_test, y_pred_knn)
print("KNN Accuracy:", knn_accuracy)


Decision Tree Accuracy: 0.9710982658959537
Random Forest Accuracy: 0.9624277456647399
SVM Accuracy: 0.8988439306358381
Logistic Regression Accuracy: 0.684971098265896
KNN Accuracy: 0.8815028901734104


In [26]:
# Confusion Matrix 계산 및 출력
print("\n=== Confusion Matrices ===")
print("Decision Tree:\n", confusion_matrix(y_test, y_pred_dt))
print("Random Forest:\n", confusion_matrix(y_test, y_pred_rf))
print("SVM:\n", confusion_matrix(y_test, y_pred_svm))
print("Logistic Regression:\n", confusion_matrix(y_test, y_pred_lr))
print("KNN:\n", confusion_matrix(y_test, y_pred_knn))


=== Confusion Matrices ===
Decision Tree:
 [[ 73   2   0   2]
 [  2  12   0   1]
 [  1   0 236   0]
 [  0   2   0  15]]
Random Forest:
 [[ 72   1   3   1]
 [  2  10   0   3]
 [  1   0 236   0]
 [  2   0   0  15]]
SVM:
 [[ 61   0  16   0]
 [ 11   2   0   2]
 [  1   0 236   0]
 [  5   0   0  12]]
Logistic Regression:
 [[ 12   0  64   1]
 [  2   0  13   0]
 [ 13   0 224   0]
 [ 11   0   5   1]]
KNN:
 [[ 56   1  20   0]
 [ 12   1   2   0]
 [  0   0 237   0]
 [  5   0   1  11]]
